In [9]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import matplotlib.pyplot as plt
import pandas as pd
from src.model import BasicModel

plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

class LEARN_INFO:
    def __init__(self, name, columns, is_contain_cluster_label=False, matching_type="general", jump=3):
        self.name = name
        self.columns = columns
        self.is_label = is_contain_cluster_label
        self.matching = matching_type
        self.jump = jump
    

# INPUT_REPORT_SIMULATION = [
#     LEARN_INFO(name="univariate", columns=['energy (kw 15min)'], is_contain_cluster_label=False),
#     LEARN_INFO(name="with cluster label", columns=['energy (kw 15min)'], is_contain_cluster_label=True),
# ]

DEFAULT_COLUMNS = [
    'energy (kw 15min)',
    ['week sin',
    'week cos'],
    ['year sin',
    'year cos'],
    'season idx',
    'weather idx',
    'avg ta',
    'avg rhm'
]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
        'energy (kw 15min)',
        'week sin',
        'week cos'
    ],
#     "multivariate (year)": [
#         'energy (kw 15min)',
#         'year sin',
#         'year cos',
#     ],
    "multivariate (weather)": [
        'energy (kw 15min)',
        'weather idx'
    ],
#     "multivariate (avg ta)": [
#         'energy (kw 15min)',
#         'avg ta'
#     ],
#     "multivariate (avg rhm)": [
#         'energy (kw 15min)',
#         'avg rhm'
#     ],
#     "multivariate (season)": [
#         'energy (kw 15min)',
#         'season idx'
#     ]
}

# Input Report Simulation
IRS = []

for key in _.keys():
#     IRS.append(
#         LEARN_INFO(name="{} / with cluster (new matching)".format(key), 
#                    columns=_[key], 
#                    is_contain_cluster_label=True,
#                    matching_type="split"),
#     )
    IRS.append(
        LEARN_INFO(name="{} / with cluster".format(key), 
                   columns=_[key], 
                   is_contain_cluster_label=True),
    )
    
    
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )

# Training Model Array 
TMA = []

is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch,
                   matching_type=_.matching)
    
    bm.set_window()
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_predict(is_reshape=True)
    bm.set_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (multivariate (weather)) set predict (validation) info start ###### 

0 / 91
50 / 91
91 / 91 complete.

###### [Notice] set predict info success ###### 



In [11]:
test_evaluate_df

,cos,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate / with cluster,0.955346,82.059432,0.194303,1.139094,0.948106,0.072787,1456.910039,261.377934
multivariate (week) / with cluster,0.955406,82.241094,0.193541,1.126872,0.948166,0.072050,1456.910039,258.731290
multivariate (weather) / with cluster,0.953169,81.703221,0.195173,1.149614,0.945009,0.074232,1456.910039,266.567610
univariate,0.953850,83.906491,0.176004,1.085071,0.943564,0.065293,1456.910039,234.467950
multivariate (week),0.955609,84.518693,0.177702,1.064328,0.944777,0.062809,1456.910039,225.548711
multivariate (weather),0.953851,83.820404,0.177357,1.088381,0.943528,0.065642,1456.910039,235.722152


In [12]:
val_evaluate_df

,cos,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate / with cluster,0.953464,82.012222,0.197642,1.205484,0.936554,0.078361,832.494756,149.747311
multivariate (week) / with cluster,0.953060,82.198373,0.195995,1.186865,0.935702,0.077550,832.494756,148.197614
multivariate (weather) / with cluster,0.951684,81.390231,0.199659,1.223727,0.934192,0.081070,832.494756,154.925354
univariate,0.942075,78.418317,0.193700,1.286621,0.922764,0.094017,832.494756,179.666381
multivariate (week),0.942357,78.531900,0.203106,1.277114,0.923070,0.093522,832.494756,178.720809
multivariate (weather),0.942277,78.502914,0.194450,1.285770,0.923131,0.093648,832.494756,178.962113


In [ ]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
        'energy (kw 15min)',
        'week sin',
        'week cos'
    ],
#     "multivariate (year)": [
#         'energy (kw 15min)',
#         'year sin',
#         'year cos',
#     ],
    "multivariate (weather)": [
        'energy (kw 15min)',
        'weather idx'
    ],
#     "multivariate (avg ta)": [
#         'energy (kw 15min)',
#         'avg ta'
#     ],
#     "multivariate (avg rhm)": [
#         'energy (kw 15min)',
#         'avg rhm'
#     ],
#     "multivariate (season)": [
#         'energy (kw 15min)',
#         'season idx'
#     ]
}

# Input Report Simulation
IRS = []

for key in _.keys():
    for jump in [3,4,6,8,12,24]:
        IRS.append(
            LEARN_INFO(name="{} / new matching {}".format(key,jump), 
                       columns=_[key], 
                       is_contain_cluster_label=True,
                       matching_type="split",
                      jump=jump),
        )


# Training Model Array 
TMA = []

is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch,
                   matching_type=_.matching,
                   jump=_.jump)
    
    bm.set_window()
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_predict(is_reshape=True)
    bm.set_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

In [16]:
test_evaluate_df

,cos,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate / new matching 3,0.947776,80.972225,0.199507,1.184353,0.937859,0.077198,1456.910039,277.217564
univariate / new matching 4,0.942318,79.007673,0.206410,1.224333,0.936528,0.085168,1456.910039,305.839322
univariate / new matching 6,0.944054,80.006930,0.202181,1.209986,0.935752,0.081114,1456.910039,291.281040
univariate / new matching 8,0.944733,80.423428,0.196497,1.180426,0.938655,0.079424,1456.910039,285.213049
univariate / new matching 12,0.944457,80.327861,0.198547,1.195672,0.937353,0.079812,1456.910039,286.605362
univariate / new matching 24,0.946709,80.966741,0.196480,1.179611,0.938244,0.077220,1456.910039,277.297466


In [17]:
val_evaluate_df

,cos,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate / new matching 3,0.944933,79.451698,0.200170,1.284768,0.924293,0.089515,832.494756,171.063538
univariate / new matching 4,0.941894,78.364767,0.209446,1.323900,0.921760,0.094250,832.494756,180.112177
univariate / new matching 6,0.942175,78.425389,0.210518,1.327944,0.919697,0.093986,832.494756,179.607505
univariate / new matching 8,0.942970,78.733428,0.201126,1.298782,0.922847,0.092644,832.494756,177.043101
univariate / new matching 12,0.943520,78.820460,0.205251,1.305616,0.922000,0.092265,832.494756,176.318558
univariate / new matching 24,0.942701,78.649675,0.204086,1.308149,0.921578,0.093009,832.494756,177.740332


In [73]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
        'energy (kw 15min)',
        'week sin',
        'week cos',
    ],
    "multivariate (week - year)": [
        'energy (kw 15min)',
        'week sin',
        'week cos',
        'year sin',
        'year cos',
    ],
    "multivariate (week - weather)": [
        'energy (kw 15min)',
        'week sin',
        'week cos',
        'weather idx',
    ],
    "multivariate (week - avg ta)": [
        'energy (kw 15min)',
        'week sin',
        'week cos',
        'avg ta'
    ],
    "multivariate (week - avg rhm)": [
        'energy (kw 15min)',
        'week sin',
        'week cos',
        'avg rhm'
    ],
    "multivariate (week - season)": [
        'energy (kw 15min)',
        'week sin',
        'week cos',
        'season idx'
    ]
}

# Input Report Simulation
IRS = []
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )

# Training Model Array 
TMA = []

is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch)
    
    bm.set_window()
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_predict(is_reshape=True)
    bm.set_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (multivariate (week - season) / with cluster) set predict (validation) info start ###### 

0 / 91
50 / 91
91 / 91 complete.

###### [Notice] set predict info success ###### 



In [74]:
test_evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,83.951244,0.175063,1.083647,0.943540,0.065112,1456.910039,233.815937
multivariate (week),84.507257,0.176618,1.063804,0.944541,0.062856,1456.910039,225.715322
multivariate (week - year),78.557215,0.224212,1.282300,0.926830,0.086996,1456.910039,312.402083
multivariate (week - weather),84.553392,0.178101,1.064973,0.944625,0.062669,1456.910039,225.043183
multivariate (week - avg ta),74.712701,0.230061,1.352023,0.925818,0.102593,1456.910039,368.413195
multivariate (week - avg rhm),84.242127,0.178447,1.074098,0.943881,0.063932,1456.910039,229.578030
multivariate (week - season),81.284910,0.193905,1.186546,0.935530,0.075929,1456.910039,272.662024
univariate / with cluster,82.797681,0.188910,1.116772,0.948542,0.069792,1456.910039,250.622318
multivariate (week) / with cluster,83.014755,0.190520,1.107923,0.947269,0.068911,1456.910039,247.459743
multivariate (week - year) / with cluster,78.523518,0.228957,1.298472,0.928559,0.087133,1456.910039,312.893016


In [75]:
val_evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,78.112268,0.193611,1.289997,0.922628,0.095350,832.494756,182.214224
multivariate (week),78.220224,0.203754,1.282021,0.922640,0.094880,832.494756,181.315495
multivariate (week - year),74.230511,0.249521,1.457535,0.913370,0.112260,832.494756,214.529645
multivariate (week - weather),78.313720,0.206137,1.282161,0.922729,0.094473,832.494756,180.537145
multivariate (week - avg ta),76.680595,0.212529,1.331393,0.918201,0.101587,832.494756,194.132827
multivariate (week - avg rhm),77.727214,0.206274,1.296736,0.921117,0.097028,832.494756,185.419776
multivariate (week - season),76.962730,0.203058,1.324650,0.918864,0.100358,832.494756,191.784064
univariate / with cluster,81.935457,0.195970,1.202640,0.935682,0.078695,832.494756,150.386375
multivariate (week) / with cluster,81.876783,0.198506,1.194899,0.934755,0.078951,832.494756,150.874830
multivariate (week - year) / with cluster,72.314580,0.264748,1.520807,0.909201,0.120607,832.494756,230.479668


In [82]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (weather)": [
        'energy (kw 15min)',
        'weather idx'
    ],
    "multivariate (weather - week)": [
        'energy (kw 15min)',
        'weather idx',
        'week sin',
        'week cos'
    ],
    "multivariate (weather - year)": [
        'energy (kw 15min)',
        'weather idx',
        'year sin',
        'year cos'
    ],
    "multivariate (weather - avg ta)": [
        'energy (kw 15min)',
        'weather idx',
        'avg ta'
    ],
    "multivariate (weather - avg rhm)": [
        'energy (kw 15min)',
        'weather idx',
        'avg rhm'
    ],
    "multivariate (weather - season)": [
        'energy (kw 15min)',
        'weather idx',
        'season idx'
    ],
}

# Input Report Simulation
IRS = []
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )

# Training Model Array 
TMA = []

is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch)
    
    bm.set_window()
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_predict(is_reshape=True)
    bm.set_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (multivariate (weather - season)) set predict (validation) info start ###### 

0 / 91
50 / 91
91 / 91 complete.

###### [Notice] set predict info success ###### 



In [83]:
test_evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,84.175705,0.173177,1.076140,0.944172,0.064201,1456.910039,230.545738
multivariate (weather),84.195381,0.173362,1.076363,0.943976,0.064121,1456.910039,230.259080
multivariate (weather - week),84.541997,0.179633,1.068108,0.944521,0.062715,1456.910039,225.209192
multivariate (weather - year),73.394104,0.243267,1.413928,0.914573,0.107943,1456.910039,387.623965
multivariate (weather - avg ta),49.173200,0.374122,1.997248,0.885861,0.206210,1456.910039,740.500747
multivariate (weather - avg rhm),83.022022,0.183861,1.120179,0.941544,0.068882,1456.910039,247.353861
multivariate (weather - season),81.446558,0.187329,1.168500,0.936068,0.075273,1456.910039,270.306954


In [84]:
val_evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,77.610628,0.195105,1.299258,0.921495,0.097535,832.494756,186.390345
multivariate (weather),78.193561,0.193008,1.285664,0.922701,0.094996,832.494756,181.537460
multivariate (weather - week),78.044941,0.207869,1.290364,0.922538,0.095643,832.494756,182.774712
multivariate (weather - year),69.414488,0.288685,1.615028,0.895602,0.133241,832.494756,254.622785
multivariate (weather - avg ta),74.691234,0.223283,1.405066,0.913758,0.110253,832.494756,210.694150
multivariate (weather - avg rhm),77.760566,0.209189,1.319386,0.922744,0.096882,832.494756,185.142125
multivariate (weather - season),75.489818,0.206412,1.367396,0.916731,0.106774,832.494756,204.045976


In [6]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (avg rhm)": [
        'energy (kw 15min)',
        'avg rhm'
    ],
    "multivariate (avg rhm - week)": [
        'energy (kw 15min)',
        'avg rhm',
        'week sin',
        'week cos'
    ],
    "multivariate (avg rhm - year)": [
        'energy (kw 15min)',
        'avg rhm',
        'year sin',
        'year cos'
    ],
    "multivariate (avg rhm - weather)": [
        'energy (kw 15min)',
        'avg rhm',
        'weather idx',
    ],
    "multivariate (avg rhm - avg ta)": [
        'energy (kw 15min)',
        'avg rhm',
        'avg ta',
    ],
    "multivariate (avg rhm - season)": [
        'energy (kw 15min)',
        'avg rhm',
        'season idx'
    ],
}

# Input Report Simulation
IRS = []
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )

# Training Model Array 
TMA = []

is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch)
    
    bm.set_window()
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_predict(is_reshape=True)
    bm.set_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] columns validation start ###### 

###### [Notice] columns validation success ###### 

connect KETIDB,,,
connect success
###### [Notice] multivariate (avg rhm - year) model Init Start ###### 

###### [Notice] jg datas load start ###### 



CursorNotFound: cursor id 4808395246158881762 not found, full error: {'ok': 0.0, 'errmsg': 'cursor id 4808395246158881762 not found', 'code': 43, 'codeName': 'CursorNotFound'}

In [ ]:
test_evaluate_df

In [ ]:
val_evaluate_df

In [3]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
        'energy (kw 15min)',
        'week sin',
        'week cos',
    ],
    "multivariate (week - weather)": [
        'energy (kw 15min)',
        'week sin',
        'week cos',
        'weather idx'
    ],
    "multivariate (week - weather - year)": [
        'energy (kw 15min)',
        'week sin',
        'week cos',
        'year sin',
        'year cos',
        'weather idx'
    ],
    "multivariate (week - weather - avg ta)": [
        'energy (kw 15min)',
        'week sin',
        'week cos',
        'avg ta',
        'weather idx'
    ],
    "multivariate (week - weather - avg rhm)": [
        'energy (kw 15min)',
        'week sin',
        'week cos',
        'avg rhm',
        'weather idx'
    ],
     "multivariate (week - weather - season)": [
        'energy (kw 15min)',
        'week sin',
        'week cos',
        'weather idx',
        'season idx',
    ],
}

# Input Report Simulation
IRS = []
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )

# Training Model Array 
TMA = []

is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch)
    
    bm.set_window()
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_predict(is_reshape=True)
    bm.set_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (multivariate (week - weather - season)) set predict (validation) info start ###### 

0 / 91
50 / 91
91 / 91 complete.

###### [Notice] set predict info success ###### 



In [4]:
test_evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,84.245569,0.174090,1.075270,0.944422,0.063918,1456.910039,229.527883
multivariate (week),84.787616,0.175486,1.056122,0.945311,0.061718,1456.910039,221.630756
multivariate (week - weather),84.455278,0.179854,1.070379,0.944299,0.063067,1456.910039,226.472614
multivariate (week - weather - year),79.026391,0.218659,1.269206,0.927888,0.085092,1456.910039,305.566619
multivariate (week - weather - avg ta),75.455039,0.229824,1.346032,0.928149,0.099582,1456.910039,357.598001
multivariate (week - weather - avg rhm),84.162301,0.180690,1.079690,0.943982,0.064255,1456.910039,230.741033
multivariate (week - weather - season),80.638038,0.201345,1.212853,0.933257,0.078554,1456.910039,282.086374


In [5]:
val_evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,78.175961,0.195414,1.289986,0.922602,0.095073,832.494756,181.683979
multivariate (week),78.375702,0.202918,1.279361,0.922925,0.094203,832.494756,180.021150
multivariate (week - weather),78.140964,0.206090,1.287973,0.921929,0.095225,832.494756,181.975331
multivariate (week - weather - year),73.956349,0.254495,1.463806,0.911162,0.113455,832.494756,216.812030
multivariate (week - weather - avg ta),76.535974,0.213222,1.337049,0.917741,0.102217,832.494756,195.336786
multivariate (week - weather - avg rhm),77.776251,0.208858,1.303286,0.921898,0.096814,832.494756,185.011545
multivariate (week - weather - season),76.446261,0.208427,1.348516,0.917203,0.102608,832.494756,196.083640


In [6]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
        'energy (kw 15min)',
        'week sin',
        'week cos'
    ],
    "multivariate (year)": [
        'energy (kw 15min)',
        'year sin',
        'year cos',
    ],
    "multivariate (weather)": [
        'energy (kw 15min)',
        'weather idx'
    ],
    "multivariate (avg ta)": [
        'energy (kw 15min)',
        'avg ta'
    ],
    "multivariate (avg rhm)": [
        'energy (kw 15min)',
        'avg rhm'
    ],
    "multivariate (season)": [
        'energy (kw 15min)',
        'season idx'
    ]
}

# Input Report Simulation
IRS = []
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )
    
for key in _.keys():
    IRS.append(
        LEARN_INFO(name="{} / with cluster".format(key), 
                   columns=_[key], 
                   is_contain_cluster_label=True),
    )
    

# Training Model Array 
TMA = []

is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch)
    
    bm.set_window()
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_predict(is_reshape=True)
    bm.set_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (multivariate (season) / with cluster) set predict (validation) info start ###### 

0 / 91
50 / 91
91 / 91 complete.

###### [Notice] set predict info success ###### 



In [7]:
test_evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,84.165976,0.172832,1.077136,0.943893,0.064240,1456.910039,230.687488
multivariate (week),84.711575,0.176005,1.056876,0.945401,0.062027,1456.910039,222.738600
multivariate (year),77.043603,0.237315,1.346469,0.922054,0.093137,1456.910039,334.454057
multivariate (weather),83.896153,0.175710,1.085676,0.943669,0.065335,1456.910039,234.618569
multivariate (avg ta),73.498045,0.240080,1.389968,0.919390,0.107521,1456.910039,386.109646
multivariate (avg rhm),83.320716,0.181035,1.108319,0.942392,0.067670,1456.910039,243.002167
multivariate (season),80.394018,0.193839,1.205136,0.935061,0.079544,1456.910039,285.641518
univariate / with cluster,82.429232,0.192499,1.128673,0.948258,0.071287,1456.910039,255.990279
multivariate (week) / with cluster,83.046792,0.189244,1.104895,0.949643,0.068781,1456.910039,246.992996
multivariate (year) / with cluster,81.799869,0.198090,1.169634,0.938508,0.073840,1456.910039,265.159542


In [8]:
val_evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,77.891495,0.194154,1.293865,0.921955,0.096312,832.494756,184.052147
multivariate (week),78.571570,0.202289,1.273224,0.923350,0.093349,832.494756,178.390556
multivariate (year),71.588913,0.271561,1.551360,0.907644,0.123768,832.494756,236.520811
multivariate (weather),78.235913,0.194538,1.287011,0.922759,0.094812,832.494756,181.184884
multivariate (avg ta),75.070819,0.213840,1.377544,0.913000,0.108600,832.494756,207.534125
multivariate (avg rhm),77.448478,0.205795,1.324183,0.920882,0.098242,832.494756,187.740235
multivariate (season),75.835909,0.204902,1.359235,0.916804,0.105267,832.494756,201.164788
univariate / with cluster,81.402927,0.197768,1.217300,0.934215,0.081015,832.494756,154.819656
multivariate (week) / with cluster,82.047816,0.198462,1.191455,0.935067,0.078206,832.494756,149.450991
multivariate (year) / with cluster,78.059597,0.206771,1.307775,0.922502,0.095580,832.494756,182.652704


In [57]:
from itertools import combinations, chain
from functools import reduce
import numpy as np

all_columns = []

for length in range(1,len(DEFAULT_COLUMNS)):
    all_columns += list(combinations(DEFAULT_COLUMNS, length))
    
IRS = []

def one_dimension(columns):
    one_col = []
    
    for col in columns:
        if str(type(col)) == "<class 'list'>":
            for _ in col:
                one_col.append(_)
        else:
            one_col.append(col)
            
    return one_col

for all_column in all_columns:
    if 'energy (kw 15min)' not in all_column:
        continue
    _ = one_dimension(list(all_column))
    IRS.append(
        LEARN_INFO(name=reduce(lambda acc, cur : acc + " - " + cur ,_,""),
                   columns=_,
                   is_contain_cluster_label=False),
    )
    
for all_column in all_columns:
    if 'energy (kw 15min)' not in all_column:
        continue
    _ = one_dimension(list(all_column))
    IRS.append(
        LEARN_INFO(name="{} / with cluster".format(reduce(lambda acc, cur : acc + " - " + cur ,_,"")), 
                   columns=_, 
                   is_contain_cluster_label=True),
    )

In [58]:
# Training Model Array 
TMA = []
total = len(IRS)
cnt = 1
is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch)

    bm.set_window()
    bm.set_model()
    
    print(bm)
    print("{} / {}".format(cnt, total))     
    bm.training(epochs=50)
    
    bm.set_predict(is_reshape=True)
    bm.set_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
    cnt += 1
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] ( - energy (kw 15min) - year sin - year cos - season idx - weather idx - avg ta - avg rhm / with cluster) set predict (validation) info start ###### 

0 / 91
50 / 91
91 / 91 complete.

###### [Notice] set predict info success ###### 



In [59]:
test_evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
- energy (kw 15min),84.237622,0.173896,1.075648,0.944393,0.063950,1456.910039,229.643666
- energy (kw 15min) - week sin - week cos,84.776281,0.175979,1.056818,0.945338,0.061764,1456.910039,221.795892
- energy (kw 15min) - year sin - year cos,74.613345,0.247820,1.406148,0.914254,0.102997,1456.910039,369.860724
- energy (kw 15min) - season idx,81.651486,0.183742,1.156001,0.937725,0.074442,1456.910039,267.321336
- energy (kw 15min) - weather idx,83.761605,0.176026,1.088257,0.943163,0.065881,1456.910039,236.578813
...,...,...,...,...,...,...,...
- energy (kw 15min) - week sin - week cos - year sin - year cos - season idx - weather idx - avg rhm / with cluster,80.470584,0.205497,1.187860,0.938430,0.079233,1456.910039,284.526019
- energy (kw 15min) - week sin - week cos - year sin - year cos - season idx - avg ta - avg rhm / with cluster,82.450476,0.194281,1.136293,0.940821,0.071200,1456.910039,255.680781
- energy (kw 15min) - week sin - week cos - year sin - year cos - weather idx - avg ta - avg rhm / with cluster,81.299790,0.200563,1.179198,0.939038,0.075869,1456.910039,272.445232
- energy (kw 15min) - week sin - week cos - season idx - weather idx - avg ta - avg rhm / with cluster,79.524523,0.214335,1.252208,0.927356,0.083071,1456.910039,298.309282


In [60]:
val_evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
- energy (kw 15min),77.987186,0.195256,1.291934,0.922191,0.095895,832.494756,183.255525
- energy (kw 15min) - week sin - week cos,78.039248,0.206187,1.287920,0.922134,0.095668,832.494756,182.822105
- energy (kw 15min) - year sin - year cos,71.653514,0.269559,1.547792,0.904476,0.123487,832.494756,235.983012
- energy (kw 15min) - season idx,76.100846,0.199084,1.341090,0.917815,0.104113,832.494756,198.959202
- energy (kw 15min) - weather idx,78.204518,0.193023,1.283462,0.923131,0.094948,832.494756,181.446245
...,...,...,...,...,...,...,...
- energy (kw 15min) - week sin - week cos - year sin - year cos - season idx - weather idx - avg rhm / with cluster,75.994638,0.213190,1.366238,0.918417,0.104575,832.494756,199.843379
- energy (kw 15min) - week sin - week cos - year sin - year cos - season idx - avg ta - avg rhm / with cluster,75.816236,0.215008,1.372861,0.921859,0.105352,832.494756,201.328565
- energy (kw 15min) - week sin - week cos - year sin - year cos - weather idx - avg ta - avg rhm / with cluster,76.533668,0.223995,1.364214,0.921398,0.102227,832.494756,195.355981
- energy (kw 15min) - week sin - week cos - season idx - weather idx - avg ta - avg rhm / with cluster,78.317506,0.226583,1.348980,0.925154,0.094456,832.494756,180.505626


In [68]:
test_evaluate_df.sort_values(by=['ecv'],ascending=True)

,ecv,mae,mean dis,mean sim,mse,tss,wss
- energy (kw 15min) - avg ta - avg rhm,53.798870,0.350022,1.888960,0.891475,0.187443,1456.910039,673.108901
- energy (kw 15min) - weather idx - avg ta - avg rhm,58.482919,0.324547,1.794656,0.898838,0.168440,1456.910039,604.866522
- energy (kw 15min) - week sin - week cos - weather idx - avg ta,70.981665,0.252254,1.461934,0.918667,0.117731,1456.910039,422.771041
- energy (kw 15min) - week sin - week cos - avg ta - avg rhm / with cluster,71.584538,0.246871,1.432211,0.921206,0.115285,1456.910039,413.987716
- energy (kw 15min) - weather idx - avg ta - avg rhm / with cluster,72.151918,0.265151,1.475232,0.920741,0.112983,1456.910039,405.721507
...,...,...,...,...,...,...,...
- energy (kw 15min),84.237622,0.173896,1.075648,0.944393,0.063950,1456.910039,229.643666
- energy (kw 15min) - week sin - week cos - weather idx - avg rhm,84.267306,0.179806,1.077254,0.943959,0.063829,1456.910039,229.211205
- energy (kw 15min) - week sin - week cos - avg rhm,84.373830,0.178002,1.068662,0.944756,0.063397,1456.910039,227.659239
- energy (kw 15min) - week sin - week cos - weather idx,84.726639,0.177485,1.060450,0.944466,0.061966,1456.910039,222.519135


In [67]:
val_evaluate_df.sort_values(by=['ecv'],ascending=True)

,ecv,mae,mean dis,mean sim,mse,tss,wss
- energy (kw 15min) - year sin - year cos - season idx - avg rhm,68.975224,0.286539,1.625074,0.896134,0.135154,832.494756,258.279634
- energy (kw 15min) - year sin - year cos - weather idx - avg rhm,69.548494,0.289295,1.607589,0.901842,0.132657,832.494756,253.507189
- energy (kw 15min) - week sin - week cos - year sin - year cos - season idx - avg ta - avg rhm,69.549295,0.286417,1.600566,0.898892,0.132653,832.494756,253.500521
- energy (kw 15min) - year sin - year cos - season idx - weather idx,69.812569,0.278195,1.595154,0.900266,0.131506,832.494756,251.308782
- energy (kw 15min) - year sin - year cos - season idx - weather idx - avg rhm,70.413868,0.263565,1.559059,0.903365,0.128887,832.494756,246.302996
...,...,...,...,...,...,...,...
- energy (kw 15min) - week sin - week cos - avg ta / with cluster,81.210724,0.198784,1.224618,0.934923,0.081852,832.494756,156.419734
- energy (kw 15min) / with cluster,81.432489,0.197487,1.218691,0.934233,0.080886,832.494756,154.573560
- energy (kw 15min) - week sin - week cos - weather idx / with cluster,81.974651,0.196879,1.190665,0.934130,0.078524,832.494756,150.060087
- energy (kw 15min) - weather idx / with cluster,82.364548,0.194552,1.191169,0.937535,0.076826,832.494756,146.814213


In [72]:
test_evaluate_df.to_csv("test_evaluate_result.csv")

In [71]:
val_evaluate_df.to_csv("val_evaluate_result.csv")

In [51]:
_ = {
    "univariate": ['energy (kw 15min)'],
}

# Input Report Simulation
IRS = []
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )
    
for key in _.keys():
    IRS.append(
        LEARN_INFO(name="{} / with cluster".format(key), 
                   columns=_[key], 
                   is_contain_cluster_label=True),
    )
    

# Training Model Array 
TMA = []

is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch)
    
    bm.set_window()
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_predict(is_reshape=True)
    bm.set_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (univariate / with cluster) set predict (validation) info start ###### 

0 / 91
50 / 91
91 / 91 complete.

###### [Notice] set predict info success ###### 



In [52]:
# TMA[0] : non-cluster
# TMA[1] : with-cluster

In [58]:
test_evaluate_df

,cos,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,0.951501,83.215000,0.173414,1.098712,0.941863,0.068099,1456.910039,244.542347
univariate / with cluster,0.951200,82.463762,0.180882,1.117995,0.943399,0.071147,1456.910039,255.487215


In [59]:
val_evaluate_df

,cos,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,0.938759,76.799998,0.188785,1.308206,0.919763,0.101067,832.494756,193.138804
univariate / with cluster,0.940480,77.724918,0.186174,1.282679,0.921922,0.097038,832.494756,185.438891


In [53]:
og_pattern = TMA[0].get_original_pattern(is_reshape=True).copy()
non_cluster_predict = TMA[0].predicts_list.copy()
cluster_predict = TMA[1].predicts_list.copy()

In [54]:
og_pattern

array([[-0.77439534, -0.82943672, -0.84778384, ..., -0.77439534,
        -0.90282522, -0.97621373],
       [-1.10464361, -1.12299074, -1.14133786, ..., -0.75604821,
        -0.8844781 , -0.99456085],
       [-1.17803212, -1.21472637, -1.25142062, ..., -0.81108959,
        -0.99456085, -1.01290798],
       ...,
       [-0.11389879, -0.11389879, -0.16894017, ...,  0.58329201,
         0.5099035 ,  0.32643224],
       [ 0.32643224,  0.36312649,  0.27139086, ...,  0.58329201,
         0.47320925,  0.01453109],
       [-0.09555167, -0.00381603, -0.05885741, ..., -0.35241143,
        -0.44414707, -0.49918844]])

In [55]:
non_cluster_predict

array([[-0.84978724, -0.8407613 , -0.81717706, ..., -0.71865845,
        -0.83580101, -0.93477154],
       [-1.12967658, -1.10650539, -1.12464237, ..., -0.82471293,
        -0.80316472, -0.89459139],
       [-1.2433604 , -1.21758783, -1.234375  , ..., -0.92837155,
        -0.85507977, -0.99720603],
       ...,
       [-0.20942007, -0.1342003 , -0.21433124, ...,  0.59275174,
         0.44846898,  0.38501835],
       [ 0.19464903,  0.20125607,  0.1994572 , ...,  0.70569229,
         0.51007462,  0.3407715 ],
       [-0.08752491,  0.00448168, -0.01398823, ..., -0.54457414,
        -0.39958093, -0.45165324]])

In [56]:
cluster_predict

array([[-0.84034073, -0.78605586, -0.768628  , ..., -0.7891013 ,
        -0.90233105, -0.98959923],
       [-1.1080718 , -1.06504381, -1.06481481, ..., -0.87748802,
        -0.89584053, -0.96186543],
       [-1.21041334, -1.17560518, -1.16198325, ..., -0.91910934,
        -0.90307021, -1.03380466],
       ...,
       [-0.22759418, -0.15082058, -0.24083711, ...,  0.37806275,
         0.30605134,  0.27579093],
       [ 0.13663135,  0.16315736,  0.13286172, ...,  0.37192193,
         0.30440748,  0.25995952],
       [-0.12796183, -0.02198093, -0.06796961, ..., -0.56139201,
        -0.44043013, -0.47775841]])

In [57]:
# MAE
from time import sleep
mae = 0
mse = 0

cluster_mae = 0
cluster_mse = 0

non_cluster = non_cluster_predict.flatten()
cluster = cluster_predict.flatten()
for idx,_ in enumerate(og_pattern[:,3:].flatten()):
    sleep(0.01)
    mae += (abs(_ - non_cluster[idx]))
    mse += ((_ - non_cluster[idx]) ** 2)
    print("[non_cluster {}] now mae: {}, mse: {}".format(
        idx + 1,
        mae / (idx + 1),
        mse / (idx + 1)
    ))
    
    cluster_mae += (abs(_ - cluster[idx]))
    cluster_mse += ((_ - cluster[idx]) ** 2)
    print("[cluster {}] now mae: {}, mse: {}".format(
        idx + 1,
        cluster_mae / (idx + 1),
        cluster_mse / (idx + 1)
    ))

[non_cluster 1] now mae: 0.002003391144427602, mse: 4.013576077570937e-06
[cluster 1] now mae: 0.007443110587444957, mse: 5.5399895216935205e-05
[non_cluster 2] now mae: 0.006663988610918159, mse: 6.612991295110513e-05
[cluster 2] now mae: 0.025411982754056817, mse: 0.0009686492344325194
[non_cluster 3] now mae: 0.018703231845921226, mse: 0.0006541784159885218
[cluster 3] now mae: 0.018863767748449296, mse: 0.0006568535511467885
[non_cluster 4] now mae: 0.027645281163509006, mse: 0.0012324179594836618
[cluster 4] now mae: 0.024746767437176298, mse: 0.0009419904177118896
[non_cluster 5] now mae: 0.05001194063275631, mse: 0.0048767891402067805
[cluster 5] now mae: 0.03093328105702462, mse: 0.0013736300153249135
[non_cluster 6] now mae: 0.10610232760097976, mse: 0.028968023918864588
[cluster 6] now mae: 0.08616562233491541, mse: 0.023024873869526962
[non_cluster 7] now mae: 0.14702453473224295, mse: 0.046844250314927095
[cluster 7] now mae: 0.12312417996401274, mse: 0.036726910210090766
[

[non_cluster 69] now mae: 0.13904656489580014, mse: 0.04016500523453436
[cluster 69] now mae: 0.13319588646522768, mse: 0.038392727224241495
[non_cluster 70] now mae: 0.14095790900179794, mse: 0.04065467689626996
[cluster 70] now mae: 0.13525252719235756, mse: 0.03894166075441424
[non_cluster 71] now mae: 0.13918009242757548, mse: 0.040085132986352814
[cluster 71] now mae: 0.13359359244962332, mse: 0.038397484358292835
[non_cluster 72] now mae: 0.13834857704105927, mse: 0.03961575936540304
[cluster 72] now mae: 0.13284838907300964, mse: 0.037952939236935074
[non_cluster 73] now mae: 0.13662686691883133, mse: 0.03907527458318014
[cluster 73] now mae: 0.1315962270664308, mse: 0.03745656089434698
[non_cluster 74] now mae: 0.13647236790669884, mse: 0.038759034691758634
[cluster 74] now mae: 0.13291879846393934, mse: 0.03766194357729924
[non_cluster 75] now mae: 0.1347856523316785, mse: 0.03824357256220595
[cluster 75] now mae: 0.13266741196315074, mse: 0.03733326141075853
[non_cluster 76] 

[non_cluster 140] now mae: 0.1379626306649221, mse: 0.040516169023065984
[cluster 140] now mae: 0.13656585928132037, mse: 0.037972291878802064
[non_cluster 141] now mae: 0.13796695247618837, mse: 0.040365006128304404
[cluster 141] now mae: 0.13615768775512002, mse: 0.03774726257961012
[non_cluster 142] now mae: 0.1374192242668341, mse: 0.04010625806780729
[cluster 142] now mae: 0.1357083325731116, mse: 0.037518298858992626
[non_cluster 143] now mae: 0.1365429551118818, mse: 0.039826820727136
[cluster 143] now mae: 0.13485025161002032, mse: 0.03725711545182286
[non_cluster 144] now mae: 0.13718344966651005, mse: 0.03991370128671723
[cluster 144] now mae: 0.13530346996530226, mse: 0.037276479169759846
[non_cluster 145] now mae: 0.13679233049010922, mse: 0.03968309375408092
[cluster 145] now mae: 0.13494212632024383, mse: 0.03706680581565166
[non_cluster 146] now mae: 0.13764789288661966, mse: 0.0398803959483334
[cluster 146] now mae: 0.13566133748858653, mse: 0.037207269762974206
[non_cl

[non_cluster 209] now mae: 0.14524727817150415, mse: 0.046072613144877095
[cluster 209] now mae: 0.15047608493769699, mse: 0.047002814197221944
[non_cluster 210] now mae: 0.1446018381873849, mse: 0.045853668247639846
[cluster 210] now mae: 0.15049021239565563, mse: 0.046891108932265946
[non_cluster 211] now mae: 0.14399357628430473, mse: 0.04563760508681822
[cluster 211] now mae: 0.14986987757856127, mse: 0.04667069677138766
[non_cluster 212] now mae: 0.14335004402994167, mse: 0.04542260329521268
[cluster 212] now mae: 0.14992937712556176, mse: 0.046575084896354715
[non_cluster 213] now mae: 0.14354526133164505, mse: 0.045369913122718666
[cluster 213] now mae: 0.1499633960741466, mse: 0.046472404265416246
[non_cluster 214] now mae: 0.14294580695760137, mse: 0.04515899263810708
[cluster 214] now mae: 0.14927601003622099, mse: 0.046255281794699295
[non_cluster 215] now mae: 0.14289432502060695, mse: 0.045029841943860886
[cluster 215] now mae: 0.14906263996159386, mse: 0.04608987052411699

[non_cluster 280] now mae: 0.13868415660497702, mse: 0.04219865926084774
[cluster 280] now mae: 0.14689298791616853, mse: 0.043868798218179995
[non_cluster 281] now mae: 0.13848610967488212, mse: 0.04207302159081693
[cluster 281] now mae: 0.14681213234319088, mse: 0.043767552771192195
[non_cluster 282] now mae: 0.1380435693644963, mse: 0.041924491049856676
[cluster 282] now mae: 0.14641733884484956, mse: 0.04361681271298966
[non_cluster 283] now mae: 0.13776483758616467, mse: 0.041788716164796715
[cluster 283] now mae: 0.14608851335603096, mse: 0.043472750690351586
[non_cluster 284] now mae: 0.13742725820249885, mse: 0.04164775224936216
[cluster 284] now mae: 0.14588614708672726, mse: 0.04334732862034771
[non_cluster 285] now mae: 0.13699548015799137, mse: 0.04150234438784246
[cluster 285] now mae: 0.1456778929212855, mse: 0.043221506705733354
[non_cluster 286] now mae: 0.13670881070149052, mse: 0.04136781130180145
[cluster 286] now mae: 0.14524044237992825, mse: 0.043071861588076674
[

...

[non_cluster 1127] now mae: 0.1399491435342853, mse: 0.040420111733301005
[cluster 1127] now mae: 0.14331661382939398, mse: 0.0404975740499394
[non_cluster 1128] now mae: 0.13982595036431836, mse: 0.04038427916497205
[cluster 1128] now mae: 0.14329597601087699, mse: 0.04047444580917757
[non_cluster 1129] now mae: 0.1398013250968638, mse: 0.04035962469433925
[cluster 1129] now mae: 0.14321783962456536, mse: 0.04044128315207865
[non_cluster 1130] now mae: 0.13984494837001502, mse: 0.040355551712241894
[cluster 1130] now mae: 0.14321023016166926, mse: 0.0404215318524913
[non_cluster 1131] now mae: 0.13991095905203468, mse: 0.04036055259469366
[cluster 1131] now mae: 0.14322443473363913, mse: 0.04040822255562164
[non_cluster 1132] now mae: 0.13979386835760566, mse: 0.04032494630513189
[cluster 1132] now mae: 0.14312393130116463, mse: 0.040373292650949674
[non_cluster 1133] now mae: 0.13976022319531983, mse: 0.04029847908141779
[cluster 1133] now mae: 0.14306073527205812, mse: 0.04034217369

[non_cluster 1199] now mae: 0.13694868626508522, mse: 0.03906894326658925
[cluster 1199] now mae: 0.14092661125669645, mse: 0.03917695359742695
[non_cluster 1200] now mae: 0.13690086192365405, mse: 0.039041660572457546
[cluster 1200] now mae: 0.14082817882138687, mse: 0.03914473962832266
[non_cluster 1201] now mae: 0.13679812508457254, mse: 0.03900930500660601
[cluster 1201] now mae: 0.14071840513814918, mse: 0.03911221346696257
[non_cluster 1202] now mae: 0.13673349325588846, mse: 0.03897975822286834
[cluster 1202] now mae: 0.14061945604721618, mse: 0.039080068895030254
[non_cluster 1203] now mae: 0.13685573812084081, mse: 0.039014304618177384
[cluster 1203] now mae: 0.14071438015275925, mse: 0.03910155657173345
[non_cluster 1204] now mae: 0.13721174141084874, mse: 0.039247491918463905
[cluster 1204] now mae: 0.14102360445164994, mse: 0.03928767651459018
[non_cluster 1205] now mae: 0.1371321168643465, mse: 0.03921633444063765
[cluster 1205] now mae: 0.14098174604124206, mse: 0.0392618

[non_cluster 1268] now mae: 0.13629072295857922, mse: 0.038834204126012364
[cluster 1268] now mae: 0.14063120263792142, mse: 0.039020874520739914
[non_cluster 1269] now mae: 0.13623152919992274, mse: 0.03880655088327692
[cluster 1269] now mae: 0.14068755519965317, mse: 0.03902558973720062
[non_cluster 1270] now mae: 0.13613897717208495, mse: 0.03877626960940423
[cluster 1270] now mae: 0.14062927383802892, mse: 0.03899836086262524
[non_cluster 1271] now mae: 0.13614917659884027, mse: 0.038763252513072666
[cluster 1271] now mae: 0.14064252462974683, mse: 0.03898718758525447
[non_cluster 1272] now mae: 0.13611631404849683, mse: 0.038739776329866166
[cluster 1272] now mae: 0.14064288499752783, mse: 0.03897218937763511
[non_cluster 1273] now mae: 0.13612923141550953, mse: 0.03872762771601919
[cluster 1273] now mae: 0.14054449296098528, mse: 0.03894176098585058
[non_cluster 1274] now mae: 0.13605716442719182, mse: 0.038698770785281104
[cluster 1274] now mae: 0.14048559496126387, mse: 0.03891

[non_cluster 1341] now mae: 0.13452700589666247, mse: 0.0379493627980154
[cluster 1341] now mae: 0.13897530154380333, mse: 0.038126328913810154
[non_cluster 1342] now mae: 0.13446374505486372, mse: 0.03792292007746069
[cluster 1342] now mae: 0.13888449838091138, mse: 0.03809813716896543
[non_cluster 1343] now mae: 0.13439645129913205, mse: 0.03789612994495488
[cluster 1343] now mae: 0.1388210205680877, mse: 0.03807191114281391


KeyboardInterrupt: 

In [48]:
og_pattern[:,3:].shape

(171, 21)

In [49]:
non_cluster_predict.shape

(171, 21)